# Imports

In [69]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, solo_encabezado, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from nlppen.extraccion.utils.extraerFechaRecibido import ExtraerFecha
from pyspark.sql.functions import length
from nlppen.extraccion.utils.misc import splitResolucion

import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configuracion Spark

In [70]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "1")
         .config("spark.executor.memory", "6g")
         .config("spark.executor.cores", "1")
         .config("spark.driver.memory", "12g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [71]:
ejecutar = False
if ejecutar:
    terminos = {
        'seguimiento': [r'\bseguimiento\b'],
        'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
        'se condena': [r'\bse condena\b', r'\bse le condena\b', r'\bse les condena\b'],
        'plan': [r'\bplan\b'],
        'plazo': [r'\bplazo\b']
    }
    seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructuralesNormal')
    print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
    seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True)
    print("Cantidad elementos despues de filtrados : " + str(seleccion.sdf.count()))
    estructurales = SentenciasEstructurales(seleccion)

# Procesamiento de todas las variables de sentencias estructurales

## La siguiente celda ejecuta el cálculo de todas las variables definidas para las sentencias estructurales

Nota: Modificar la bandera "ejecutar" para aplicar todo el procesamiento.

In [72]:
from pyspark.sql.types import *

if ejecutar:
    columnas = {
        'se ordena PER' : ArrayType(StringType()),
        'se ordena LOC' : ArrayType(StringType()),
        'se ordena ORG' : ArrayType(StringType()),
        'se ordena MISC' : ArrayType(StringType()),
        'se ordena GPE' : ArrayType(StringType()),
        'se ordena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeOrdena(columnas, True, True)

    columnas = {
        'se condena PER' : ArrayType(StringType()),
        'se condena LOC' : ArrayType(StringType()),
        'se condena ORG' : ArrayType(StringType()),
        'se condena MISC' : ArrayType(StringType()),
        'se condena GPE' : ArrayType(StringType()),
        'se condena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeCondena(columnas, True, True)

    columnas = {
        'extension sentencia' : IntegerType(),
        'extension por lo tanto' : IntegerType()
    }
    estructurales.extraerExtension(columnas, True)

    columnas = {
        'plazosDefinidos' : ArrayType(TimestampType())
    }
    estructurales.plazosDefinidos(columnas, True)

    columnas = {
        'FechaSolicitud' : TimestampType(),
    }

    estructurales.extrarFechaRecibido(columnas, True)

    columnas = {
        'num resolucion' : StringType()
    }

    estructurales.extraerNumeroSentencia(columnas, True)

    columnas = {
        'inst internacionales' : ArrayType(StringType())
    }

    estructurales.extraerInstrumentosInternacionales(columnas, True)

    columnas = {
        'derechos Norm' : ArrayType(StringType()),
        'derechos GenXPat' : ArrayType(StringType())
    }
    estructurales.extraerDerechos(columnas, True)

    columnas = {
        'derechos Acotados' : ArrayType(StringType()),
        'derechos General' : ArrayType(StringType()),
        'derechos Fundamental' : ArrayType(StringType()),
        'derechos Humano' : ArrayType(StringType())
    }

    estructurales.extraerDerechosSinNormalizar(columnas, True)
    
    sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
    columnas = {
        'id_sentencia' : StringType(),
        'num_doc_oficial' : StringType(),
        'expediente_oficial' : StringType()
    }
    estructurales.agregarIDSentencia(columnas, sentencias, True)
    
    columnas = {
        'citasVotDate' : MapType(StringType(), TimestampType()),
        'citasIDVoto' : MapType(StringType(), StringType())
    }
    estructurales.extrarCitaSentenciasFecha(columnas, sentencias, True)
    #Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
    estructurales.seleccion.guardarDatos()

## Guardar los resultados del preprocesamiento por año

In [5]:
if ejecutar:
    df = spark.read.parquet(seleccion.datasets_path+"/terminos.parquet")
    writer = df.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')


## Cargar los resultados preprocesados y divididos por año

In [73]:
if not ejecutar:
    seleccion = Seleccion({}, spark, parquet_path='./dataset/procesado', datasets_path='./datasets/estructuralesNormal')
    estructurales = SentenciasEstructurales(seleccion)

In [74]:
estructurales.seleccion.sdf.toPandas().head(5)

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,Fechahora_ents,Redactor_ents,CitaSentencias_ents,Reglamento_ents,termino_lst,Cantón_ents,Ley_ents,EntidadPublica_ents,Constitución_ents,Fecha_ents,seguimiento,se_ordena,se_condena,plan,plazo,se_ordena_PER,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,se_ordena_Ent_Pub,se_condena_PER,se_condena_LOC,se_condena_ORG,se_condena_MISC,se_condena_GPE,se_condena_Ent_Pub,extension_sentencia,extension_por_lo_tanto,plazosDefinidos,FechaSolicitud,num_resolucion,inst_internacionales,derechos_Norm,derechos_GenXPat,derechos_Acotados,derechos_General,derechos_Fundamental,derechos_Humano,id_sentencia,num_doc_oficial,expediente_oficial,citasVotDate,citasIDVoto,anno
0,2018112102451287_19152.html,*180169790007CO* Exp: 18-016979-0007-CO Res. Nº 2018019152 SALA CONSTITUCION...,SALUD,0.994687,180169790007CO,2018-11-16 09:15:00,Araya García,Recurso de Amparo,Con lugar,False,[las nueve horas quince minutos del dieciseis de noviembre de dos mil diecio...,[Araya García],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],"[Puriscal, Puriscal, Puriscal]","[Ley General de la Administración Pública, Ley General de la Administración ...","[Caja Costarricense de Seguro Social, Caja Costarricense del Seguro Social, ...",[21],"[26 de octubre de 2018, 30 de octubre de 2018, 2 de noviembre de 2018, 11 de...",0,1,1,1,2,"[Ileana Balmaceda Arias, Eugenia Cruz Harley, DEYDAMIA, UMAÑA SANDÍ]",[Hospital San Juan de Dios],"[Directora General y Jefa, Servicio de Oftalmología, Servicio de Oftalmologí...",[TRES MES],None,None,None,None,[Caja Costarricense de Seguro Social],None,None,[Caja Costarricense de Seguro Social],18471,2641,[1970-04-01 00:00:00],NaT,2018019152,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho fundamental a la salud, derecho a la vida, dere...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho a ...","[derecho en una parte de la población, derecho fundamental a la salud, derec...",[derecho fundamental a la salud],[derecho humano que tiene el derecho a la salud de todos los ciudadanos],sen-1-0007-893680,19152,180169790007CO,None,None,2018
1,2018112063619_17421.html,*180157550007CO* Exp: 18-015755-0007-CO Res. Nº 2018017421 SALA CONSTITUCION...,SALUD,1.000000,180157550007CO,2018-10-19 09:30:00,Fernández Argüello,Recurso de Amparo,Con lugar,False,[las nueve horas treinta minutos del diecinueve de octubre de dos mil diec...,[Fernández Argüello],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],"[Alajuela, Jiménez, JIMÉNEZ, Jiménez, Jiménez, Jiménez]","[Ley de la Jurisdicción, Ley de la Jurisdicción]","[Caja Costarricense del Seguro Social, Caja Costarricense de Seguro Social, ...",[21],"[07 de octubre del 2018, 08 de octubre del 2018, 07 de setiembre del 2018, 0...",0,1,1,0,0,"[Gustavo Jiménez Ramírez, Jefe del Servicio, Douglas Montero Chacón]",[Hospital México],"[Director General, INMEDIATA]",[Valor 002] ],None,None,None,None,[Caja Costarricense del Seguro],None,None,[Caja Costarricense de Seguro Social],17177,2252,None,2018-10-07 00:00:00,2018017421,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho a la salud, derecho a la salud, derecho a la sa...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho hu...","[derecho a la igualdad de trato ante, derecho en una parte de la población, ...",None,[derecho humano que tiene el derecho a],sen-1-0007-860153,17421,180157550007CO,None,None,2018
2,201812701524278_20041.html,*180178880007CO* Exp: 18-017888-0007-CO Res. Nº 2018020041 SALA CONSTITUCION...,TRABAJO,0.632804,180178880007CO,2018-11-30 09:30:00,Fernández Argüello,Recurso de Amparo,Con lugar parcial,False,[las nueve horas treinta minutos del treinta de noviembre de dos mil diecioc...,[Fernández A

## Ejecutar un criterio

In [58]:
from pyspark.sql.types import *
if not ejecutar:
    sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
    columnas = {
        'citasVotDate' : MapType(StringType(), TimestampType()),
        'citasIDVoto' : MapType(StringType(), StringType())
    }
    estructurales.extrarCitaSentenciasFecha(columnas, sentencias, True)
    s = estructurales.seleccion.sdf

In [60]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)
#s.toPandas()

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,Fechahora_ents,Redactor_ents,CitaSentencias_ents,Reglamento_ents,termino_lst,Cantón_ents,Ley_ents,EntidadPublica_ents,Constitución_ents,Fecha_ents,seguimiento,se_ordena,se_condena,plan,plazo,se_ordena_PER,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,se_ordena_Ent_Pub,se_condena_PER,se_condena_LOC,se_condena_ORG,se_condena_MISC,se_condena_GPE,se_condena_Ent_Pub,extension_sentencia,extension_por_lo_tanto,plazosDefinidos,FechaSolicitud,num_resolucion,inst_internacionales,derechos_Norm,derechos_GenXPat,derechos_Acotados,derechos_General,derechos_Fundamental,derechos_Humano,id_sentencia,num_doc_oficial,expediente_oficial,citasVotDate,citasIDVoto,anno,id_sentenciax,num_doc_oficialx,expediente_oficialx
0,2018112102451287_19152.html,*180169790007CO* Exp: 18-016979-0007-CO Res. Nº 2018019152 SALA CONSTITUCION...,SALUD,0.994687,180169790007CO,2018-11-16 09:15:00,Araya García,Recurso de Amparo,Con lugar,False,[las nueve horas quince minutos del dieciseis de noviembre de dos mil diecio...,[Araya García],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],"[Puriscal, Puriscal, Puriscal]","[Ley General de la Administración Pública, Ley General de la Administración ...","[Caja Costarricense de Seguro Social, Caja Costarricense del Seguro Social, ...",[21],"[26 de octubre de 2018, 30 de octubre de 2018, 2 de noviembre de 2018, 11 de...",0,1,1,1,2,"[Ileana Balmaceda Arias, Eugenia Cruz Harley, DEYDAMIA, UMAÑA SANDÍ]",[Hospital San Juan de Dios],"[Directora General y Jefa, Servicio de Oftalmología, Servicio de Oftalmologí...",[TRES MES],None,None,None,None,[Caja Costarricense de Seguro Social],None,None,[Caja Costarricense de Seguro Social],18471,2641,[1970-04-01 00:00:00],NaT,2018019152,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho fundamental a la salud, derecho a la vida, dere...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho a ...","[derecho en una parte de la población, derecho fundamental a la salud, derec...",[derecho fundamental a la salud],[derecho humano que tiene el derecho a la salud de todos los ciudadanos],sen-1-0007-893680,19152,180169790007CO,None,None,2018,sen-1-0007-893680,19152,180169790007CO
1,2018112063619_17421.html,*180157550007CO* Exp: 18-015755-0007-CO Res. Nº 2018017421 SALA CONSTITUCION...,SALUD,1.000000,180157550007CO,2018-10-19 09:30:00,Fernández Argüello,Recurso de Amparo,Con lugar,False,[las nueve horas treinta minutos del diecinueve de octubre de dos mil diec...,[Fernández Argüello],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],"[Alajuela, Jiménez, JIMÉNEZ, Jiménez, Jiménez, Jiménez]","[Ley de la Jurisdicción, Ley de la Jurisdicción]","[Caja Costarricense del Seguro Social, Caja Costarricense de Seguro Social, ...",[21],"[07 de octubre del 2018, 08 de octubre del 2018, 07 de setiembre del 2018, 0...",0,1,1,0,0,"[Gustavo Jiménez Ramírez, Jefe del Servicio, Douglas Montero Chacón]",[Hospital México],"[Director General, INMEDIATA]",[Valor 002] ],None,None,None,None,[Caja Costarricense del Seguro],None,None,[Caja Costarricense de Seguro Social],17177,2252,None,2018-10-07 00:00:00,2018017421,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho a la salud, derecho a la salud, derecho a la sa...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho hu...","[derecho a la igualdad de trato ante, derecho en una parte de la población, ...",None,[derecho humano que tiene el derecho a],sen-1-0007-860153,17421,180157550007CO,None,None,2018,sen-1-0007-860153,17421,180157550007CO
2,201812701524278_20041.html,*180178880007CO* Exp: 18-017888-0007-CO Res. Nº 2018020041 SALA CONSTITUCION...,TRABAJO,0.632804,180178880007CO,2018-11-30 09:30:00,Fernández Argüello,Recur

In [61]:
s = estructurales.seleccion.sdf
dateVoto = s.filter(s.citasVotDate.isNotNull()).count()
IDVoto = s.filter(s.citasIDVoto.isNotNull()).count()
print(dateVoto)
print(IDVoto)
print("Acc", IDVoto/dateVoto)

452
438
Acc 0.9690265486725663


In [18]:
#Expediente 2772C93
#['2772', 'C', '93']

sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
x = sentencias[sentencias.numeroDocumento == 6762].sort_values(by="anno")
x
x[x.anno == 1997]

,Unnamed: 0,anno,expediente,id,numeroDocumento,sentenciasRelacionadas
264370,264370,1997,970062670007CO,sen-1-0007-112335,6762,NaN


In [24]:
splitResolucion("6762",None, sentencias, 1997)

6762


('sen-1-0007-112335', '6762', '970062670007CO')

In [63]:
from pyspark.sql.functions import explode,posexplode, desc
pd.set_option('display.max_rows', 29)
pd.set_option('display.max_columns', None)
newData = s.filter(s.citasVotDate.isNotNull() & s.citasIDVoto.isNull())
print("Faltantes", newData.count())
#newData.toPandas()[['expediente', 'id_sentencia', 'num_resolucion', 'num_doc_oficial']].head(50)

nuevo = newData.select('id_sentencia', 'anno' ,explode('citasVotDate').alias('voto', "Fecha"),'citasIDVoto')
nuevo.toPandas()

Faltantes 14


,id_sentencia,anno,voto,Fecha,citasIDVoto
0,sen-1-0007-735968,2018,77-2017,NaT,None
1,sen-1-0007-717298,2017,2015-011362,2015-07-28 14:50:00,None
2,sen-1-0007-635514,2015,28-2015,NaT,None
3,sen-1-0007-448264,2008,07265-99,1999-09-17 12:00:00,None
4,sen-1-0007-404360,2008,65-01,2001-04-03 16:00:00,None
5,sen-1-0007-424137,2008,11262-2005,2005-08-24 15:00:00,None
6,sen-1-0007-621621,2014,201400793014095-2008,2008-09-23 09:00:00,None
7,sen-1-0007-271989,2004,2002-11706,2002-12-13 10:00:00,None
8,sen-1-0007-201581,2002,5084-98,1998-07-15 18:00:00,None
9,sen-1-0007-187879,1998,0504-I-97,NaT,None


In [51]:
for i in range(11, 12):
    num_resolucion = nuevo.toPandas().iloc[i, 1]
    fecha  = nuevo.toPandas().iloc[i, 2]
    print(fecha)
    print(num_resolucion)
    print("******************************")
    print(splitResolucion(num_resolucion, None, sentencias, fecha))

1997-10-17 11:00:00
6762
******************************
1997
('sen-1-0007-112335', '6762', '970062670007CO')


In [65]:
#writer = estructurales.seleccion.sdf.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')

In [ ]:
"""from pyspark.sql.functions import explode,posexplode, desc
print("Sentencias con citas (antiguas)", s.filter(s.CitaSentencias_ents.isNotNull()).count())
print("Sentencias con citas (nuevas)", s.filter(s.citasFechas.isNotNull()).count())

print("Citas que no fueron encontradas previamente", s.filter(s.CitaSentencias_ents.isNull() & s.citasFechas.isNotNull()).count())
print("Citas que no fueron encontradas ahora en comparación con las antiguas", s.filter(s.CitaSentencias_ents.isNotNull() & s.citasFechas.isNull()).count())

totalAntiguas = (s.filter(s.CitaSentencias_ents.isNotNull())
    .select(explode('CitaSentencias_ents').alias('CitaSentencias_ents')))
totalNuevas = (s.filter(s.citasFechas.isNotNull())
    .select(explode('citasFechas')))

totalConFecha = totalNuevas.filter(totalNuevas.value.isNotNull())
print("Total de citas (antiguas)", totalAntiguas.count())
print("Total de citas (nuevas)", totalNuevas.count())
print("Total de citas con fecha (nuevas)", totalConFecha.count())
print("Porcentaje de extracción de fechas ", totalConFecha.count()/totalNuevas.count())
"""


# Cargar procesamiento de sentencias estructurales.

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)

# Zona de trabajo (ignorar)

In [ ]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)


In [ ]:
import re 
txt = "resolucion Nº 878 de las 10:02 horas del 17 de marzo de 2004."
meses = "(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|septiembre|setiembre|octubre|noviembre|diciembre)"
sentenciaRe = "(?:[Ss]entencia|[Rr]esoluci[oó]n)\s*(?:estructural)?\s*"
numeroRe    = "(?:(?:n[uú]mero)|(?:[nN][ºo°][\.|,]?))"
sentenciaCitadaRe ="\s*(?P<sentencia_citada>[\d]+\s?[\-\—]?[a-z]?\s?[\-\—]?[\d]+)[\.|,]?"
horasRe = "(?:(?:\s?del?\s?las?\s?(?P<hora>[\w|\d]+)?\s?(?:(?:\:|horas?|hrs.?))?\s?)?"
minutosRe = "(?:(?P<minutos>[\w|\s|\d]+)\s+(?:(?:horas?|hrs.?|minutos?)?))?"
diaRe = "\s?del?\s?(?:d[íi]a)?(?P<dia>[\w|\s|\d]+)\s+de"
mesRe = "\s+(?P<mes>"+meses+")+\s+"
a = "(?:mil\s?novecientos\s?[\w]+\s?y?\s?[\w]+)|(?:dos\s?mil\s?[\w]+)"
annoRe = "(?:del?)?\s?(?:año)?\s?(?P<anno>(?:(?:[\d][\d][\d][\d])|(?:"+a+")))?)?"

#Union de todas las expresiones regulares.
sentenciaCitadaRe = sentenciaRe+numeroRe+sentenciaCitadaRe+horasRe+minutosRe+diaRe+mesRe+annoRe

citaSentenciaExp = re.compile(
        sentenciaCitadaRe,  re.X | re.M | re.I)
matches = re.finditer(citaSentenciaExp, txt)
for match in matches:
    a , b = match.span(1) 
    print(a, b)
    print(txt[a:b])

In [68]:
#sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
#sentencias.head(5)

In [ ]:
"""citas = newData.toPandas()
#citas = s.filter(s.CitaSentencias_ents.isNotNull()).toPandas()
nlp = spark_get_spacy("es_core_news_lg")
#size = len(citas)
size = 1
for i in range(0, 10):
    sent = citas.iloc[i, 4]
    txt = citas.iloc[i, 1]
    num_resolucion = citas.iloc[i, -13]
    citasExtraidas  = citas.iloc[i, -2]
    print("******************************")
    print(sent)
    comprobarNumResolucion(num_resolucion, sentencias, sent)"""